<a href="https://colab.research.google.com/github/tahaozdenoglu/Colab/blob/main/aractahminmodeli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Araç Fiyat Tahmin Çalışması (Regresyon)

## 1.kütüphanaleri içeri aktar yükle

In [ ]:
#kütüphane versiyonlarını göster
import sklearn
print(sklearn.__version__)
import pandas
print(pandas.__version__)
import numpy
print(numpy.__version__)
import matplotlib
print(matplotlib.__version__)
import seaborn
print(seaborn.__version__)
import scipy
print(scipy.__version__)
import gradio
print(gradio.__version__)
import xlrd
print(xlrd.__version__)


1.6.1
2.2.2
2.0.2
3.10.0
0.13.2
1.15.3
5.31.0
2.0.2


In [ ]:
import pandas as pd ##veri seti yükleme
from sklearn.model_selection import train_test_split ##veri setini bölme
from sklearn.linear_model import LinearRegression ##model olusturma
from sklearn.metrics import mean_squared_error, r2_score ## modeli değerlendirme
from sklearn.pipeline import Pipeline ##standart iş akısı
from sklearn.compose import ColumnTransformer ##özellik dönüştürme işlemleri
from sklearn.preprocessing import OneHotEncoder, StandardScaler##ön işlemde standartlaştıma ve kategorik verilerin sayısal eşlenmesi için

## 2. Veri Yükleme

In [ ]:
#/content/drive/MyDrive/Colab Notebooks/Adsız klasör/cars.xls

In [ ]:
df=pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Adsız klasör/cars.xls")

In [ ]:
df

,Price,Mileage,Make,Model,Trim,Type,Cylinder,Liter,Doors,Cruise,Sound,Leather
0,17314.103129,8221,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,1
1,17542.036083,9135,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
2,16218.847862,13196,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,1,0
3,16336.913140,16342,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,0
4,16339.170324,19832,Buick,Century,Sedan 4D,Sedan,6,3.1,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
799,16507.070267,16229,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,0,0
800,16175.957604,19095,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0
801,15731.132897,20484,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0
802,15118.893228,25979,Saturn,L Series,L300 Sedan 4D,Sedan,6,3.0,4,1,1,0


## 3. Keşifsel Veri Analizi ve Veri Ön İşleme Aşaması

In [ ]:
## veri seti hakkında bilgi
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 804 entries, 0 to 803
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Price     804 non-null    float64
 1   Mileage   804 non-null    int64  
 2   Make      804 non-null    object 
 3   Model     804 non-null    object 
 4   Trim      804 non-null    object 
 5   Type      804 non-null    object 
 6   Cylinder  804 non-null    int64  
 7   Liter     804 non-null    float64
 8   Doors     804 non-null    int64  
 9   Cruise    804 non-null    int64  
 10  Sound     804 non-null    int64  
 11  Leather   804 non-null    int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 75.5+ KB


In [ ]:
# Veri setinin Özellikler ve hedef-sonuç ayrılması x de price ayırıyorsun y de price tanımladın
X=df.drop("Price",axis=1)
y=df["Price"]

In [ ]:
# Veri setinin bölünmesi x ve y yi %20si test olacak şekilde aldık test size 0.2 %20 demek randım_state her zaman 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train,X_test

(     Mileage       Make        Model                 Trim         Type  \
 344    20000  Chevrolet     Corvette              Conv 2D  Convertible   
 350     7054  Chevrolet     Corvette             Coupe 2D        Coupe   
 443    15903  Chevrolet       Malibu             Sedan 4D        Sedan   
 331     6709  Chevrolet       Cobalt             Sedan 4D        Sedan   
 290    13776  Chevrolet       Cobalt             Coupe 2D        Coupe   
 ..       ...        ...          ...                  ...          ...   
 71      9795      Buick  Park Avenue  Special Ed Ultra 4D        Sedan   
 106    24415   Cadillac      Deville         DHS Sedan 4D        Sedan   
 270     4922  Chevrolet     Cavalier             Sedan 4D        Sedan   
 435    18721  Chevrolet       Malibu        MAXX Hback 4D    Hatchback   
 102    16041   Cadillac      Deville         DHS Sedan 4D        Sedan   
 
      Cylinder  Liter  Doors  Cruise  Sound  Leather  
 344         8    6.0      2       1      0

In [ ]:
y_train,y_test #y kısmına fiyatları tanımladın geri kalanı x de

(344    41371.379273
 350    39875.854255
 443    17162.478255
 331    14696.028990
 290    14023.938547
            ...     
 71     26060.335350
 106    35866.582709
 270    14061.122765
 435    17173.942322
 102    39307.009165
 Name: Price, Length: 643, dtype: float64,
 192    11302.902985
 717    29114.544506
 168    10777.052537
 522    20677.593819
 536    15253.868566
            ...     
 658    29142.714386
 356    35575.417049
 525    18042.221074
 748    14881.955446
 517    19204.809341
 Name: Price, Length: 161, dtype: float64)

In [ ]:
## sütun dönüşüm işlemleri one hot katogiriği sayısala çeviriyor
##veri ön işleyici
preprocessor=ColumnTransformer(transformers=[('num', StandardScaler(),
                                             ['Mileage','Cylinder','Liter','Doors']),
                                             ('cat',OneHotEncoder(),
                                             ['Make','Model','Trim','Type'])])

In [ ]:
## model seç
model=LinearRegression()

In [ ]:
# iş hattı olustur pipline iş hattıdır stepsler adımları gösteriri bu bu işlemleri yap diye
## adımları belirt : veri ön işleme ve model olarak
pipe=Pipeline(steps=[('preprocessor',preprocessor),
                     ('model',model)])

In [ ]:
## Modeli eğit
pipe.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Mileage', 'Cylinder',
                                                   'Liter', 'Doors']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Make', 'Model', 'Trim',
                                                   'Type'])])),
                ('model', LinearRegression())])

In [ ]:
## tahminleri yapsın
y_pred=pipe.predict(X_test)
## Değerlendir
print('RMSE',mean_squared_error(y_test,y_pred)**.5) ## kalıp bunlar ortalama yapılan hata 835 dolar
print('R2',r2_score(y_test,y_pred)) ## r kare burası %99 gayet iyi


RMSE 835.1013862206736
R2 0.9912072687898575


In [ ]:
## tam veri seti ile eğitim (bölmeden) (bütün veri setini kullanarak eğit)
pipe.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Mileage', 'Cylinder',
                                                   'Liter', 'Doors']),
                                                 ('cat', OneHotEncoder(),
                                                  ['Make', 'Model', 'Trim',
                                                   'Type'])])),
                ('model', LinearRegression())])

In [ ]:
## modeli pkl olarak dısarı aktar-- dosya olarak aktardık  model dosyası bu hugginface atıcaz bunu
import joblib
joblib.dump(pipe,'arac_fiyat.pkl')

['arac_fiyat.pkl']

In [ ]:
df.columns

Index(['Price', 'Mileage', 'Make', 'Model', 'Trim', 'Type', 'Cylinder',
       'Liter', 'Doors', 'Cruise', 'Sound', 'Leather'],
      dtype='object')

In [ ]:
## hugging face app kodları
import gradio as gr
import pandas as pd
import joblib
model=joblib.load('/content/arac_fiyat.pkl')  ##solda drive orda modelin dosyası var ordan yolu kopyaladık huggin facede dosya konumu olacak
df=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Adsız klasör/cars.xls')
# Fiyat tahmin fonksiyonu yukarıda belirtilen özellikleri alıp pipeline vericek ve otomatik ön işleme yapılacak model de eğitildiği için çıktı olarak verecek
def price(Make,Model,Trim,Type,Mileage,Cylinder,Liter,Doors,Cruise,Sound,Leather):
    input_data=pd.DataFrame({'Make':[Make],
                          'Model':[Model],
                          'Trim':[Trim],
                          'Mileage':[Mileage],
                          'Type':[Type],
                          'Cylinder':[Cylinder],
                          'Liter':[Liter],
                          'Doors':[Doors],
                          'Cruise':[Cruise],
                          'Sound':[Sound],
                          'Leather':[Leather]})              ## değerler dateframe olusturulacak
    prediction=model.predict(input_data)
    return prediction[0]   ##sadece dolar olsun diye sıfır
  #gradio arayüzü olusutr gr blokları ile basla
with gr.Blocks() as demo:
  gr.Markdown('Araç Fiyat Tahmini @Taha') ##markdown=baslık
  gr.Markdown('Araba Fiyatı Tahmini için aşağıdaki formu doldurunuz')
  make=gr.Dropdown(label='Marka', choices=df['Make'].unique().tolist()) ##dropdown açılır liste makeden uniqleri seç yani distinct gibi sonra onun baslıgını marka yap to list ile listeleme
  model=gr.Dropdown(label='Model', choices=df['Model'].unique().tolist())
  trim=gr.Dropdown(label='Trim', choices=df['Trim'].unique().tolist())
  Mileage=gr.Number(label='Mileage')
  Type=gr.Dropdown(label='Type', choices=df['Type'].unique().tolist())
  Cylinder=gr.Number(label='Cylinder')
  Liter=gr.Number(label='Liter')

  if __name__ == '__main__':
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://290e63b9e049a7a1bc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
